In [1]:
## Import libraries

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import regex as re
from sklearn.preprocessing import normalize
from sklearn.feature_selection import VarianceThreshold

In [2]:
'''
Function: outcome_split

Purpose: Make the outcome variable binary, splitting it  within the percentiles
of previously inputed low and high tails. 

Input: Dataframe, outcome variable name, low tail value, high tail value

Output: Dataframe with "outcome" variable split in 1s and 0s as asked by the user

'''

def outcome_split(df, outcome, low_tail, high_tail):
    
    ## Create percentiles

    my_series = df[outcome]
    my_series = my_series.values.tolist()
    my_series = np.array(my_series)

    q1, q3 = np.percentile(my_series, [low_tail, high_tail])
    
    ## Divide by percentiles

    constraint = df[outcome] >= q3
    constraint2 = df[outcome] < q1


    df.loc[constraint, 'outcome'] = int(1)
    df.loc[constraint2, 'outcome'] = int(0)
    
    df.dropna(subset = ['outcome'], inplace=True)
    
    return df

In [3]:
'''
Function: clean_data

Purpose: Transform yes and no into 1 and 0, remove some useless variables, drop variables 
less that 220 entries, only keep numerical variables

Input: Dataframe with the data

Output: Clean dataframe

'''

def clean_data(df):
    
    # Replace yes or no answers for 1 and 0
    df = df.replace("sim", int(1))
    df = df.replace("Sim", int(1))
    df = df.replace("SIM", int(1))
    df = df.replace("s", int(1))
    df = df.replace("S", int(1))
    df = df.replace("yes", int(1))
    df = df.replace("Yes", int(1))
    df = df.replace("YES", int(1))
    df = df.replace("y", int(1))
    df = df.replace("Y", int(1))
    df = df.replace("nao", int(0))
    df = df.replace("Nao", int(0))
    df = df.replace("NAO", int(0))
    df = df.replace("não", int(0))
    df = df.replace("Não", int(0))
    df = df.replace("NÃO", int(0))
    df = df.replace("n", int(0))
    df = df.replace("N", int(0))
    df = df.replace("No", int(0))
    df = df.replace("no", int(0))
    df = df.replace("NO", int(0))
    
    r = re.compile(".*bks*")
    id_columns = list(filter(r.match, df.columns)) # Read Note
    df.drop(columns=id_columns, inplace=True)
    
    r = re.compile("ID_.*")
    id_columns = list(filter(r.match, df.columns)) # Read Note
    df.drop(columns=id_columns, inplace=True)
    
    r = re.compile("TMRAW.*")
    id_columns = list(filter(r.match, df.columns)) # Read Note
    df.drop(columns=id_columns, inplace=True)
    
    r = re.compile("RESP.*")
    id_columns = list(filter(r.match, df.columns)) # Read Note
    df.drop(columns=id_columns, inplace=True)
    
    r = re.compile("G05")
    id_columns = list(filter(r.match, df.columns)) # Read Note
    df.drop(columns=id_columns, inplace=True)
    
    
    df = df.apply(pd.to_numeric, errors='ignore')
    df = df.select_dtypes(include=np.number)
    
    
    
    df.dropna(thresh=220, axis=1, inplace=True)
    
    for var in df.columns:
        if df[var].nunique() == 1:
            df.drop(columns=[var], inplace=True)
    
    print("\nThe clean dataset has", df.shape[0], "rows and", df.shape[1], "columns")
    
    return df

In [4]:
'''
Function: feature_variance_finder

Purpose: Removing variables below a variance threshold

Input: Dataframe with the data and desired variance threshold

Output: Dataframe without the low variance features

'''


def feature_variance_finder(data, thresh):
   
    normalized = normalize(data)
    data_scaled = pd.DataFrame(normalized)
    
    data_scaled.var()
    
    #storing the variance and name of variables
    variance = data_scaled.var()
    columns = data.columns
    
    #saving the names of variables having variance more than a threshold value

    variable = [ ]

    for i in range(0,len(variance)):
        if variance[i]>=thresh: #setting the threshold as 1%
            variable.append(columns[i])
            
    new_data = data[variable]
    
    return new_data

In [11]:
## Open the excel file
name= "../data/Data_filtered_PROPER.csv"
df = pd.read_csv(name)

In [12]:
## Split outcome 70/30 and drop any possible missing data
df_70_30 = outcome_split(df, "CSSA5.TOTAL", 30, 70)

df_70_30.dropna(inplace=True, axis=1)

df_70_30.dropna(inplace=True)

In [13]:
## Check shape of resulting dataframe
df_70_30.shape

(240, 277)

In [14]:
## Drop uninformative columns and columns that contain direct information about outcome
df_70_30.drop(columns=["Unnamed: 0", "ID_continua.int", "CSSAimprove", "CSSA5.TOTAL"], inplace=True)

In [15]:
## Clean data converting all remaining variables into numerical values and dropping any that cannot be converted
df_70_30 = clean_data(df_70_30)


The clean dataset has 240 rows and 261 columns


In [16]:
## Remove features with low variance
df_70_30 = feature_variance_finder(df_70_30, 0.0000001)

In [17]:
## Clean data converting all remaining variables into numerical values and dropping any that cannot be converted
df_70_30 = clean_data(df_70_30)


The clean dataset has 240 rows and 228 columns


In [18]:
## Split train and test dataset 80/20 split
train_70_30, test_70_30 = train_test_split(df_70_30, test_size=0.20, random_state=42)

In [19]:
## Check shape of train dataset
train_70_30.shape

(192, 228)

In [20]:
## Check shape of test dataset
test_70_30.shape

(48, 228)

In [21]:
## Save test and train dataset
train_70_30.to_csv("../data/train_70_30.csv", index=False)
test_70_30.to_csv("../data/test_70_30.csv", index=False)